In [1]:
from autogen import AssistantAgent, GroupChatManager, GroupChat, config_list_from_json
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS

from CypherExecutor import CypherCodeExecutor

In [ ]:
# This list will 'connect' to the llama3.1 model running on llm container
config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "llama3.1"})


llm_config = {"config_list": config_list, "temperature": 0.2}

prompt = """You are a data scientist that works with Cypher queries.
All you have to do is translate the given answer as Cypher queries.

You have to respect this rules:
- You can use the pdf files. 
- You must generate the easiest query possible in cypher format.
- Every query must be in a separate cypher format.
- You must instert all the information you have and where you found them.
- You must be precise. 

Now i will give you some information about the database schema.
- nodes -
(:Movie), Describe a movie that has a title and a plot. It can also have the number of likes.
(:Person), Describe actors and directors. They have a name, a birthday.

- relationships -
(:Movie) <-[:DIRECTED]- (:Person)
(:Person) -[:ACTED-IN]-> (:Movie)
(:Person) -[:KNOWS]-> (:Person)
(:Movie) -[:type]-> (:Genre)

QUESTION IS:
{input_question}
"""

print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

In [ ]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    retrieve_config={
        "task": "code",
        "docs_path": "https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf",
        "model": config_list[0]["model"],
        "get_or_create": True,
        "customized_prompt": prompt
    },
    code_execution_config=False,
    description="Assistant who has extra content retrieval power for solving difficult problems."
)

coder    = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=prompt,
    llm_config=llm_config,
)


executor   = AssistantAgent(
    name="executor",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    description= """Executor provides feedback based on the errors and warning generated by the query.
                    If the query is successful you can sand 'TERMINATE'. """,
    code_execution_config={"executor": CypherCodeExecutor()}
    )


PROBLEM  = "How do i retrieve all movie titles?"

def _reset_agents():
    doc_retriever.reset()
    coder.reset()
    executor.reset()


def rag_chat():
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, coder, executor], messages=[], 
                          max_round=5, speaker_selection_method="round_robin")
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    doc_retriever.initiate_chat(manager,message=doc_retriever.message_generator, problem=PROBLEM,n_results=3)


In [5]:
rag_chat()

Model llama3.1 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['d1a87522', '64188057', '86b22f11']]
Adding content of doc d1a87522 to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 64188057 to context.


Model llama3.1 not found. Using cl100k_base encoding.


doc_retriever (to chat_manager):

You are a data scientist that works with Cypher queries.
All you have to do is translate the given answer as Cypher queries.

You have to respect this rules:
- You can use the pdf files. 
- You must generate the easiest query possible in cypher format.
- Every query must be in a separate cypher format.
- You must instert all the information you have and where you found them.
- You must be precise. 

Now i will give you some information about the database schema.
- nodes -
(:Movie), Describe a movie that has a title and a plot. It can also have the number of likes.
(:Person), Describe actors and directors. They have a name, a birthday.

- relationships -
(:Movie) <-[:DIRECTED]- (:Person)
(:Person) -[:ACTED-IN]-> (:Movie)
(:Person) -[:KNOWS]-> (:Person)
(:Movie) -[:type]-> (:Genre)

QUESTION IS:
How do i retrieve all movie titles?


--------------------------------------------------------------------------------

Next speaker: coder

[autogen.oai.client:

[{'title': 'Toy Story'},
 {'title': 'Jumanji'},
 {'title': 'Grumpier Old Men'},
 {'title': 'Waiting to Exhale'},
 {'title': 'Father of the Bride Part II'},
 {'title': 'Heat'},
 {'title': 'Sabrina'},
 {'title': 'Tom and Huck'},
 {'title': 'Sudden Death'},
 {'title': 'GoldenEye'},
 {'title': 'American President, The'},
 {'title': 'Dracula: Dead and Loving It'},
 {'title': 'Balto'},
 {'title': 'Nixon'},
 {'title': 'Cutthroat Island'},
 {'title': 'Casino'},
 {'title': 'Sense and Sensibility'},
 {'title': 'Four Rooms'},
 {'title': 'Ace Ventura: When Nature Calls'},
 {'title': 'Money Train'},
 {'title': 'Get Shorty'},
 {'title': 'Copycat'},
 {'title': 'Assassins'},
 {'title': 'Powder'},
 {'title': 'Leaving Las Vegas'},
 {'title': 'Othello'},
 {'title': 'Now and Then'},
 {'title': 'Persuasion'},
 {'title': 'City of Lost Children, The (Cité des enfants perdus, La)'},
 {'title': 'Shanghai Triad (Yao a yao yao dao waipo qiao)'},
 {'title': 'Dangerous Minds'},
 {'title': 'Twelve Monkeys (a.k.a. 1

executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

Next speaker: doc_retriever

doc_retriever (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: coder

[autogen.oai.client: 09-09 13:41:02] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
coder (to chat_manager):

It seems like you didn't provide the actual output of the query execution.

However, I can tell you that if the query executed successfully, it would return a list of movie titles. If you want to see the actual output, you can modify the query to include more information, for example:

```cypher
MATCH (m:Movie)
RETURN m.title AS title, m.plot AS plot
```

This query will return both the title and the plot of each movie.
